In [ ]:
# Torch - Torchvision - IceVision - IceData - MMDetection - YOLOv5 - EfficientDet Installation
!wget https://raw.githubusercontent.com/airctic/icevision/master/icevision_install.sh
    

In [ ]:
#!wget https://raw.githubusercontent.com/airctic/icevision/master/icevision_install.sh

!bash icevision_install.sh cuda11 master

In [ ]:
!pip install pydicom

In [ ]:
!pip install openpyxl

In [ ]:
# Restart kernel after installation
import IPython    
IPython.Application.instance().kernel.do_shutdown(True)

In [ ]:
from icevision.all import *

In [ ]:
import pydicom
import pandas as pd

In [ ]:
df = pd.read_excel('../input/tinytinyexcel/tiniest-tiny.xlsx');

In [ ]:
df.head()

In [ ]:
#template_record = ObjectDetectionRecord() #template recordlarımız ObjectDetectionRecord'dan gelsin.
template_record = GrayScaleObjectDetectionRecord()

In [ ]:
def split_coordinates(coordinate):
    try:
        if(math.isnan(coordinate)):
            return [214,313,227,330] #KESİNLİKLE DÜZELTİMELİ, HATA ALINMASIN DİYE BUNU YAZDIK!
    except: # except blogu sayesinde MinikDataSetteki float değerler çalışıyor, bunu değiştirmek lazım!
        coordinate = str(coordinate)
        result = coordinate.split('-')
        xmin, ymin= result[0].split(',')
        xmax, ymax = result[1].split(',')
        return [float(xmin), float(ymin), float(xmax), float(ymax)] #pythondan coklu value donmek icin list kullanilabilir.
        

In [ ]:
a,b,c,d = split_coordinates('1,2-3,4')

In [ ]:
c_m = ClassMap(list(df.sinif.unique()))

In [ ]:
data_dir = '../input/tinytiny/' #Abdominal yolu.

In [ ]:

class AbdominalParser(Parser):
    def __init__(self, template_record, df, data_dir): #datamıza bir referans geçmemiz gerekir, bu senaryoda referans data_dirdir.
        super().__init__(template_record=template_record) # #super, ard arda inheritance ya da birden cok inheritance oldugunda kullanılır. ChessParser Parser'dan Parser da ParserInterface'dan inheritance olur.
        self.data_dir = data_dir # Js'deki this mantıgı gibi,  ChessParser's data_dir = data_dir.
        self.df = df
        self.class_map = ClassMap(list(self.df.sinif.unique())) #label sütunundaki unique değerlerden bir ClassMap olusturdu, duplicate değerleri dikkate almadı.
    def __iter__(self) -> Any: #burada yield edilen her o(her bir df satırı..) nesnesi, parse_fields fonksiyonuna parametre olarak verilecektir.
        for o in self.df.itertuples(): #itertuples yardımıyla, dataframe'i iterable hale getiriyoruz.
            yield o #Generator
    def __len__(self) -> int: #döngüde kaç tane rowa ugranilacagini  gösterir.
        return len(self.df)
    def record_id(self, o) -> Hashable: #hashlanabilir data dönmelidir(int, string etc..) NOT:Bu datasetine özel, aynı isme sahip fotoğrafların tek bir kayıtta tutulmasını istyioruz. 
        return o.kesit_numarasi
    def parse_fields(self, o, record, is_new): #buradaki metodların hepsi, ChessParser class instance'i parse edilince çalıştırılacak.
            if is_new:
                filename = str(o.kesit_numarasi) + '.png'
                record.set_filepath(f'{data_dir}/{o.olgu_numarasi}/{filename}') #BURADAKİ KESİT NUMARASI NORMALDE İNTEGERDİR.
                #record.set_img_size(ImgSize(width=512, height=512, depth = 1))
                record.set_img_size(get_img_size(f'{self.data_dir}/{o.olgu_numarasi}/{filename}'))
                record.detection.set_class_map(self.class_map) # class mapimiz
                record.tip = o.tip
                # print(o.tip)
            xmin, ymin, xmax, ymax = split_coordinates(o.koordinatlar) #excel dosyasındaki koordinat degerleri aslında FLOATTIR!
            record.detection.add_bboxes([BBox.from_xyxy(xmin, ymin, xmax, ymax)])
            record.detection.add_labels([o.sinif])

In [ ]:
parser = AbdominalParser(template_record, df, data_dir)

In [ ]:
train_records, valid_records = parser.parse(autofix = True)

In [ ]:
train_records[0]

In [ ]:
#transformlar
presize = 384
image_size = 512
train_tfms = tfms.A.Adapter([*tfms.A.aug_tfms(size=image_size, presize=presize), tfms.A.Normalize()])
valid_tfms = tfms.A.Adapter([*tfms.A.resize_and_pad(size=image_size), tfms.A.Normalize()])


In [ ]:
train_ds = Dataset(train_records, train_tfms)
valid_ds = Dataset(valid_records, valid_tfms)

In [ ]:
show_samples(random.choices(train_ds, k=6), class_map=parser.class_map, ncols=3)

In [ ]:
# Just change the value of selection to try another model

selection = 1

extra_args = {}

if selection == 0:
  model_type = models.mmdet.vfnet
  backbone = model_type.backbones.resnet50_fpn_mstrain_2x

if selection == 1:
  model_type = models.mmdet.models.retinanet
  backbone = model_type.backbones.resnet101_fpn_1x

elif selection == 2:
  model_type = models.mmdet.faster_rcnn
  backbone = model_type.backbones.resnet101_fpn_1x
  # extra_args['cfg_options'] = { 
  #   'model.bbox_head.loss_bbox.loss_weight': 2,
  #   'model.bbox_head.loss_cls.loss_weight': 0.8,
  #    }

elif selection == 3:
  # The Retinanet model is also implemented in the torchvision library
  model_type = models.torchvision.retinanet
  backbone = model_type.backbones.resnet50_fpn

elif selection == 4:
  model_type = models.ross.efficientdet
  backbone = model_type.backbones.tf_d5_ap
  # The efficientdet model requires an img_size parameter
  extra_args['img_size'] = image_size

elif selection == 5:
  model_type = models.ultralytics.yolov5
  backbone = model_type.backbones.large  #medium, small
  # The yolov5 model requires an img_size parameter
  extra_args['img_size'] = image_size

elif selection == 6:
  model_type = models.mmdet.ssd
  backbone = model_type.backbones.ssd512 #ssd512, ssdlite_mobilenetv2

elif selection == 7:
  model_type = models.fastai.unet
  backbone = model_type.backbones.resnet50 #ssd512, ssdlite_mobilenetv2
  extra_args['img_size'] = image_size

model_type, backbone, extra_args


In [ ]:
# Instantiate the model
model = model_type.model(backbone=backbone(pretrained=True), num_classes=len(parser.class_map), **extra_args)

In [ ]:
# Data Loaders
train_dl = model_type.train_dl(train_ds, batch_size=2, num_workers=2, shuffle=True)
valid_dl = model_type.valid_dl(valid_ds, batch_size=2, num_workers=2, shuffle=False)


In [ ]:
len(train_ds), len(valid_ds)

In [ ]:
metrics = [COCOMetric(metric_type=COCOMetricType.bbox)]

In [ ]:
learn = model_type.fastai.learner(dls=[train_dl, valid_dl], model=model, metrics=metrics)

In [ ]:
learn.lr_find()

In [ ]:
  learn.fine_tune(20, 0.0015754399462603033, freeze_epochs=1)

In [ ]:
checkpoint_path = 'retinanet_bitirme.pth'
models.save_icevision_checkpoint(model, 
                        model_name='mmdet.retinanet', 
                        backbone_name='resnet101_fpn_1x',
                        classes =  c_m.get_classes(), 
                        img_size=image_size, 
                        filename=checkpoint_path,
                        meta={'icevision_version': '0.12.0'})

In [ ]:
x = model_type.show_results(model, valid_ds, detection_threshold=.5)

In [ ]:
infer_dl = model_type.infer_dl(valid_ds, batch_size=4, shuffle=False)
preds = model_type.predict_from_dl(model, infer_dl, keep_images=True)

In [ ]:
show_pred(pred[0].pred, figsize= (16,12) )

In [ ]:
#show_preds(preds, figsize = (512,512))

In [ ]:
preds[0].pred

In [ ]:
show_samples(preds[:4].pred, figsize = (10,8), class_map = parser.class_map)

In [ ]:
for i in preds:
    print(i.pred)